In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.stem import PorterStemmer
from sklearn.svm import SVC

ps = PorterStemmer()

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir("drive/My Drive/IR/Project/")
os.getcwd()
os.listdir()

['data_valid.csv', 'data_train.csv', 'project.ipynb']

In [6]:

df = pd.read_csv("data_train.csv")
texts = (df['title'].apply(str.lower)).to_list()
categories = df['category'].to_list()


In [7]:
# Encode labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(categories)

# Binary Vectorization for baseline model
vectorizer_baseline = CountVectorizer(binary=True)

In [8]:

X_baseline = vectorizer_baseline.fit_transform(texts)

# Train-test split
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_baseline, labels, test_size=0.2, random_state=42)

# Define the parameter grid for C
param_grid = {'C': [0.0625, 0.125, 0.25, 0.5, 1]}

# GridSearchCV with LinearSVC for baseline model
grid_search_base = GridSearchCV(LinearSVC(dual=False), param_grid)
grid_search_base.fit(X_train_base, y_train_base)

# Best model for baseline
best_svm_base = grid_search_base.best_estimator_


# Stop Word Removal and Stemming

In [9]:
df = pd.read_csv("data_train.csv")
texts = (df['title'].apply(str.lower)).to_list()
categories = df['category'].to_list()
def stem_text(text):
    tokens = text.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Apply the stemming function to the titles
texts_stemmed = [stem_text(text) for text in texts]

In [10]:
# Binary Vectorization for the second model with stop-word removal
vectorizer_second = CountVectorizer(binary=True, stop_words='english')
X_second = vectorizer_second.fit_transform(texts_stemmed)

# Train-test split
X_train_second, X_test_second, y_train_second, y_test_second = train_test_split(X_second, labels, test_size=0.2, random_state=42)

# GridSearchCV with LinearSVC for the second model
grid_search_second = GridSearchCV(LinearSVC(dual=False), param_grid)
grid_search_second.fit(X_train_second, y_train_second)

# Best model for the second model
best_svm_stemmed = grid_search_second.best_estimator_

# Evaluate the second model
y_pred_second = best_svm_stemmed.predict(X_test_second)
accuracy_second = accuracy_score(y_test_second, y_pred_second)
error_rate_second = 1 - accuracy_second
print(f'Second Model Accuracy: {accuracy_second}')
print(f'Second Model Error Rate: {error_rate_second}')

# Calculate the relative error ratio
# relative_error_ratio = error_rate_second / error_rate_base
# print(f'Relative Error Ratio (Second Model / Baseline): {relative_error_ratio}')

# # Additional evaluation metrics
# print("Baseline Model Classification Report:")
# print(classification_report(y_test_base, y_pred_base, target_names=label_encoder.classes_))
# print("Baseline Model Confusion Matrix:")
# print(confusion_matrix(y_test_base, y_pred_base))

print("Second Model Classification Report:")
second = classification_report(y_test_second, y_pred_second, target_names=label_encoder.classes_)
print(second)
print("Second Model Confusion Matrix:")
print(confusion_matrix(y_test_second, y_pred_second))

Second Model Accuracy: 0.9298608240319691
Second Model Error Rate: 0.07013917596803088
Second Model Classification Report:
                                       precision    recall  f1-score   support

                              Cameras       0.93      0.93      0.93       392
                  Computers & Laptops       0.92      0.89      0.90       594
                              Fashion       0.95      0.96      0.96      1204
                      Health & Beauty       0.92      0.94      0.93       753
                        Home & Living       0.90      0.89      0.90      1237
                      Home Appliances       0.91      0.88      0.89       293
                    Mobiles & Tablets       0.95      0.96      0.96      1431
TV, Audio / Video, Gaming & Wearables       0.85      0.85      0.85       474
         Watches Sunglasses Jewellery       0.96      0.96      0.96       879

                             accuracy                           0.93      7257
      

In [11]:
df = pd.read_csv("data_train.csv")
texts = (df['title'].apply(str.lower)).to_list()
categories = df['category'].to_list()

# One-vs-One SVM Model


In [12]:
# Binary Vectorization for One-vs-One model
vectorizer_ovo = CountVectorizer(binary=True)
X_ovo = vectorizer_ovo.fit_transform(texts)

# Train-test split
X_train_ovo, X_test_ovo, y_train_ovo, y_test_ovo = train_test_split(X_ovo, labels, test_size=0.2, random_state=42)

# Define the parameter grid for C
param_grid = {'C': [0.0625]}

# GridSearchCV with SVC for One-vs-One model
grid_search_ovo = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid,verbose=3)
grid_search_ovo.fit(X_train_ovo, y_train_ovo)

# Best model for One-vs-One
best_svm_ovo = grid_search_ovo.best_estimator_




Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..........................C=0.0625;, score=0.656 total time= 2.3min
[CV 2/5] END ..........................C=0.0625;, score=0.653 total time= 2.3min
[CV 3/5] END ..........................C=0.0625;, score=0.664 total time= 2.3min
[CV 4/5] END ..........................C=0.0625;, score=0.650 total time= 2.3min
[CV 5/5] END ..........................C=0.0625;, score=0.651 total time= 2.3min


# Crammer & Singer SVM Model


In [13]:
# Binary Vectorization for Crammer & Singer model
vectorizer_cs = CountVectorizer(binary=True)
X_cs = vectorizer_cs.fit_transform(texts)

# Train-test split
X_train_cs, X_test_cs, y_train_cs, y_test_cs = train_test_split(X_cs, labels, test_size=0.2, random_state=42)

# Define the parameter grid for C
param_grid = {'C': [0.0625, 0.125, 0.25, 0.5, 1]}

# GridSearchCV with LinearSVC for Crammer & Singer model
grid_search_cs = GridSearchCV(LinearSVC(multi_class='crammer_singer', dual=False), param_grid,verbose=3)
grid_search_cs.fit(X_train_cs, y_train_cs)

# Best model for Crammer & Singer
best_svm_cs = grid_search_cs.best_estimator_


Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END ..........................C=0.0625;, score=0.926 total time=   1.1s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END ..........................C=0.0625;, score=0.926 total time=   1.2s


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5] END ...............................C=1;, score=0.907 total time=   6.3s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END ...............................C=1;, score=0.904 total time=   3.1s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Uni-gram + Bi-gram
*  Bigram SVM One vs Rest
*  Bigram SVM One vs One
*  Bigram SVM Crammer and Singer





In [14]:
# Encode labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(categories)

# Bigram + Unigram Vectorization
vectorizer_bigram = TfidfVectorizer(ngram_range=(1, 2), norm='l2')
X_bigram = vectorizer_bigram.fit_transform(texts)

X_train_bigram, X_test_bigram, y_train_bigram, y_test_bigram = train_test_split(X_bigram, labels, test_size=0.2, random_state=42)
# Define the parameter grid for C
param_grid = {'C': [0.0625, 0.125, 0.25, 0.5, 1]}

**Bigram SVM One vs Rest**

In [15]:
# GridSearchCV with LinearSVC for bigram model
grid_search_bigram = GridSearchCV(LinearSVC(dual=False), param_grid)
grid_search_bigram.fit(X_train_bigram, y_train_bigram)

# Best model for baseline
best_svm_bigram = grid_search_bigram.best_estimator_


**Bigram SVM One vs One**

In [16]:
# GridSearchCV with LinearSVC for bigram model
param_grid = {'C': [0.0625]}
grid_search_bigram = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid,verbose=3)
grid_search_bigram.fit(X_train_bigram, y_train_bigram)

# Best model for baseline
best_svm_bigram_ovo= grid_search_bigram.best_estimator_



Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..........................C=0.0625;, score=0.383 total time= 4.3min
[CV 2/5] END ..........................C=0.0625;, score=0.383 total time= 4.3min
[CV 3/5] END ..........................C=0.0625;, score=0.380 total time= 4.3min
[CV 4/5] END ..........................C=0.0625;, score=0.386 total time= 4.3min
[CV 5/5] END ..........................C=0.0625;, score=0.383 total time= 4.3min


**Bigram SVM Crammer and Singer**

In [17]:
# GridSearchCV with LinearSVC for bigram model
param_grid = {'C': [0.0625, 0.125, 0.25, 0.5, 1]}
grid_search_bigram = GridSearchCV(LinearSVC(multi_class='crammer_singer', dual=False), param_grid,verbose=3)
grid_search_bigram.fit(X_train_bigram, y_train_bigram)

# Best model for baseline
best_svm_bigram_cs = grid_search_bigram.best_estimator_



Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END ..........................C=0.0625;, score=0.909 total time=   3.3s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END ..........................C=0.0625;, score=0.908 total time=   3.1s


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/5] END ...............................C=1;, score=0.940 total time=   7.4s
[CV 4/5] END ...............................C=1;, score=0.936 total time=   5.3s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END ...............................C=1;, score=0.933 total time=   7.2s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Polynomial
*   SVM One vs One
*   SVM One vs Rest
*   SVM Crammer and Singer

In [18]:


# Polynomial Mapping (Degree-2) using Hashing
vectorizer_unigram = TfidfVectorizer(norm='l2')
X_unigram = vectorizer_unigram.fit_transform(texts)
hashing = HashingVectorizer(n_features=2**20)  # Adjust n_features based on your memory constraints
X_poly = hashing.transform(texts)

X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_poly, labels, test_size=0.2, random_state=42)



**Polynomial Degree 2 SVM One vs One**

In [19]:
# GridSearchCV with LinearSVC for poly model
param_grid = {'C': [0.0625]}
grid_search_poly = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid,verbose=3)
grid_search_poly.fit(X_train_poly, y_train_poly)

# Best model for poly
best_svm_poly_ovo = grid_search_poly.best_estimator_



Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..........................C=0.0625;, score=0.662 total time= 2.2min
[CV 2/5] END ..........................C=0.0625;, score=0.660 total time= 2.2min
[CV 3/5] END ..........................C=0.0625;, score=0.668 total time= 2.2min
[CV 4/5] END ..........................C=0.0625;, score=0.653 total time= 2.2min
[CV 5/5] END ..........................C=0.0625;, score=0.654 total time= 2.2min


In [20]:
param_grid = {'C': [0.0625, 0.125, 0.25, 0.5, 1]}

**Polynomial Degree 2 SVM One vs Rest**

In [21]:
# GridSearchCV with LinearSVC for poly model
grid_search_poly = GridSearchCV(LinearSVC(dual=False), param_grid)
grid_search_poly.fit(X_train_poly, y_train_poly)

# Best model for poly
best_svm_poly = grid_search_poly.best_estimator_



**Polynomial Degree 2 Crammer and Singer**

In [22]:
# GridSearchCV with LinearSVC for poly model
grid_search_poly = GridSearchCV(LinearSVC(multi_class='crammer_singer', dual=False), param_grid,verbose=3)
grid_search_poly.fit(X_train_poly, y_train_poly)

# Best model for poly
best_svm_poly_cs = grid_search_poly.best_estimator_



Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END ..........................C=0.0625;, score=0.904 total time=   1.0s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END ..........................C=0.0625;, score=0.902 total time=   1.4s


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/5] END ...............................C=1;, score=0.933 total time=   7.2s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5] END ...............................C=1;, score=0.931 total time=  13.9s
[CV 5/5] END ...............................C=1;, score=0.926 total time=   3.9s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Comparing All the models

In [23]:
print("Model 0: Baseline Model ")
# Evaluate the baseline model
y_pred_base = best_svm_base.predict(X_test_base)
accuracy_base = accuracy_score(y_test_base, y_pred_base)
error_rate_base = 1 - accuracy_base
print(f'Baseline Model Accuracy: {accuracy_base}')
print(f'Baseline Model Error Rate: {error_rate_base}')

# Classification report
baseline = classification_report(y_test_base,y_pred_base, target_names=label_encoder.classes_)
print(baseline)

# Confusion matrix
print(confusion_matrix(y_test_base,y_pred_base))


print("Model 1: Baseline Model with Stop Word Removal and Stemmed Text ")
# Evaluate the second model
y_pred_second = best_svm_stemmed.predict(X_test_second)
accuracy_second = accuracy_score(y_test_second, y_pred_second)
error_rate_second = 1 - accuracy_second
print(f'Second Model Accuracy: {accuracy_second}')
print(f'Second Model Error Rate: {error_rate_second}')

# Calculate the relative error ratio
relative_error_ratio_baseline_stemmed = error_rate_second / error_rate_base
print(f'Relative Error Ratio (One vs Rest / Baseline): {relative_error_ratio_baseline_stemmed}')

print("Second Model Classification Report:")
second = classification_report(y_test_second, y_pred_second, target_names=label_encoder.classes_)
print(second)
print("Second Model Confusion Matrix:")
print(confusion_matrix(y_test_second, y_pred_second))

print("Model 2: Baseline Model with One vs One SVM ")
# Evaluate the One-vs-One model
y_pred_ovo = best_svm_ovo.predict(X_test_ovo)
accuracy_ovo = accuracy_score(y_test_ovo, y_pred_ovo)
error_rate_ovo = 1 - accuracy_ovo
print(f'One-vs-One Model Accuracy: {accuracy_ovo}')
print(f'One-vs-One Model Error Rate: {error_rate_ovo}')

# Calculate the relative error ratio
relative_error_ratio_baseline_ovo = error_rate_ovo / error_rate_base
print(f'Relative Error Ratio (One vs One / Baseline): {relative_error_ratio_baseline_ovo}')

# Additional evaluation metrics
print("One-vs-One Model Classification Report:")
ovo = classification_report(y_test_ovo, y_pred_ovo, target_names=label_encoder.classes_)
print(ovo)
print("One-vs-One Model Confusion Matrix:")
print(confusion_matrix(y_test_ovo, y_pred_ovo))

print("Model 3: Baseline Model with Crammer and Singer SVM ")
# Evaluate the Crammer & Singer model
y_pred_cs = best_svm_cs.predict(X_test_cs)
accuracy_cs = accuracy_score(y_test_cs, y_pred_cs)
error_rate_cs = 1 - accuracy_cs
print(f'Crammer & Singer Model Accuracy: {accuracy_cs}')
print(f'Crammer & Singer Model Error Rate: {error_rate_cs}')

# Calculate the relative error ratio
relative_error_ratio_baseline_cs = error_rate_cs / error_rate_base
print(f'Relative Error Ratio (Crammer and Singer / Baseline): {relative_error_ratio_baseline_cs}')

# Additional evaluation metrics
print("Crammer & Singer Model Classification Report:")
crammer_singer = classification_report(y_test_cs, y_pred_cs, target_names=label_encoder.classes_)
print(crammer_singer)
print("Crammer & Singer Model Confusion Matrix:")
print(confusion_matrix(y_test_cs, y_pred_cs))

print("Model 4: Uni-gram + Bi-gram Model with One vs Rest SVM ")
# Evaluate the baseline model
y_pred_bigram = best_svm_bigram.predict(X_test_bigram)
accuracy_bigram = accuracy_score(y_test_bigram, y_pred_bigram)
error_rate_bigram = 1 - accuracy_bigram
print(f'Unigram + Bigram SVM One vs Rest Model Accuracy: {accuracy_bigram}')
print(f'Unigram + Bigram SVM One vs Rest Model Accuracy: {error_rate_bigram}')

# Calculate the relative error ratio
relative_error_ratio_baseline_bigram = error_rate_bigram / error_rate_base
print(f'Relative Error Ratio (Bigram OvR / Baseline): {relative_error_ratio_baseline_bigram}')

# Classification report
bigram = classification_report(y_test_bigram,y_pred_bigram, target_names=label_encoder.classes_)
print(bigram)

# Confusion matrix
print(confusion_matrix(y_test_bigram,y_pred_bigram))

print("Model 5: Uni-gram + Bi-gram Model with One vs One SVM ")
# Evaluate the baseline model
y_pred_bigram = best_svm_bigram_ovo.predict(X_test_bigram)
accuracy_bigram_ovo = accuracy_score(y_test_bigram, y_pred_bigram)
error_rate_bigram_ovo = 1 - accuracy_bigram_ovo
print(f'Unigram + Bigram SVM One vs One Model Accuracy: {accuracy_bigram_ovo}')
print(f'Unigram + Bigram SVM One vs One Model Accuracy: {error_rate_bigram_ovo}')

# Calculate the relative error ratio
relative_error_ratio_baseline_bigram_ovo = error_rate_bigram_ovo / error_rate_base
print(f'Relative Error Ratio (Bigram OvO / Baseline): {relative_error_ratio_baseline_bigram_ovo}')

# Classification report
bigram_ovo = classification_report(y_test_bigram,y_pred_bigram, target_names=label_encoder.classes_)
print(bigram_ovo)

# Confusion matrix
print(confusion_matrix(y_test_bigram,y_pred_bigram))

print("Model 6: Uni-gram + Bi-gram Model with Crammer and Singer SVM ")
# Evaluate the baseline model
y_pred_bigram = best_svm_bigram_cs.predict(X_test_bigram)
accuracy_bigram_cs = accuracy_score(y_test_bigram, y_pred_bigram)
error_rate_bigram_cs = 1 - accuracy_bigram_cs
print(f'Unigram + Bigram SVM Crammer and Singer Model Accuracy: {accuracy_bigram_cs}')
print(f'Unigram + Bigram SVM Crammer and Singer Model Accuracy: {error_rate_bigram_cs}')

# Calculate the relative error ratio
relative_error_ratio_baseline_bigram_cs = error_rate_bigram_cs / error_rate_base
print(f'Relative Error Ratio (Bigram Crammer and Singer / Baseline): {relative_error_ratio_baseline_bigram_cs}')

# Classification report
bigram_cs = classification_report(y_test_bigram,y_pred_bigram, target_names=label_encoder.classes_)
print(bigram_cs)

# Confusion matrix
print(confusion_matrix(y_test_bigram,y_pred_bigram))

print("Model 9: Degree 2 Polynomial Model with OvO SVM ")
# Evaluate the poly model
y_pred_poly = best_svm_poly_ovo.predict(X_test_poly)
accuracy_poly_ovo = accuracy_score(y_test_poly, y_pred_poly)
error_rate_poly_ovo = 1 - accuracy_poly_ovo
print(f'Polynomial Degree 2 SVM One vs One Model Accuracy: {accuracy_poly_ovo}')
print(f'Polynomial Degree 2 SVM One vs One Model Accuracy: {error_rate_poly_ovo}')

# Calculate the relative error ratio
relative_error_ratio_baseline_poly_ovo = error_rate_poly_ovo / error_rate_base
print(f'Relative Error Ratio (Poly OvO / Baseline): {relative_error_ratio_baseline_poly_ovo}')

# Classification report
poly_ovo = classification_report(y_test_poly,y_pred_poly, target_names=label_encoder.classes_)
print(poly_ovo)

# Confusion matrix
print(confusion_matrix(y_test_poly,y_pred_poly))

print("Model 8: Degree 2 Polynomial Model with OvR SVM ")
# Evaluate the poly model
y_pred_poly = best_svm_poly.predict(X_test_poly)
accuracy_poly = accuracy_score(y_test_poly, y_pred_poly)
error_rate_poly = 1 - accuracy_poly
print(f'Polynomial Degree 2 SVM One vs One Model Accuracy: {accuracy_poly}')
print(f'Polynomial Degree 2 SVM One vs One Model Accuracy: {error_rate_poly}')

# Calculate the relative error ratio
relative_error_ratio_baseline_poly = error_rate_poly / error_rate_base
print(f'Relative Error Ratio (Poly OvR / Baseline): {relative_error_ratio_baseline_poly}')

# Classification report
poly = classification_report(y_test_poly,y_pred_poly, target_names=label_encoder.classes_)
print(poly)

# Confusion matrix
print(confusion_matrix(y_test_poly,y_pred_poly))

print("Model 9: Degree 2 Polynomial Model with Crammer and Singer SVM ")
# Evaluate the poly model
y_pred_poly = best_svm_poly_cs.predict(X_test_poly)
accuracy_poly_cs = accuracy_score(y_test_poly, y_pred_poly)
error_rate_poly_cs = 1 - accuracy_poly_cs
print(f'Polynomial Degree 2 SVM One vs One Model Accuracy: {accuracy_poly_cs}')
print(f'Polynomial Degree 2 SVM One vs One Model Accuracy: {error_rate_poly_cs}')

# Calculate the relative error ratio
relative_error_ratio_baseline_poly_cs = error_rate_poly_cs / error_rate_base
print(f'Relative Error Ratio (Poly Crammer and Singer / Baseline): {relative_error_ratio_baseline_poly_cs}')

# Classification report
poly_cs = classification_report(y_test_poly,y_pred_poly, target_names=label_encoder.classes_)
print(poly_cs)

# Confusion matrix
print(confusion_matrix(y_test_poly,y_pred_poly))

Model 0: Baseline Model 
Baseline Model Accuracy: 0.9315143998897616
Baseline Model Error Rate: 0.06848560011023841
                                       precision    recall  f1-score   support

                              Cameras       0.94      0.93      0.93       392
                  Computers & Laptops       0.92      0.89      0.91       594
                              Fashion       0.96      0.97      0.96      1204
                      Health & Beauty       0.93      0.94      0.93       753
                        Home & Living       0.90      0.90      0.90      1237
                      Home Appliances       0.91      0.88      0.90       293
                    Mobiles & Tablets       0.95      0.96      0.96      1431
TV, Audio / Video, Gaming & Wearables       0.85      0.85      0.85       474
         Watches Sunglasses Jewellery       0.96      0.96      0.96       879

                             accuracy                           0.93      7257
             

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Polynomial Degree 2 SVM One vs One Model Accuracy: 0.7075926691470305
Polynomial Degree 2 SVM One vs One Model Accuracy: 0.29240733085296955
Relative Error Ratio (Poly OvO / Baseline): 4.269617706237423
                                       precision    recall  f1-score   support

                              Cameras       0.93      0.59      0.72       392
                  Computers & Laptops       0.94      0.58      0.72       594
                              Fashion       0.94      0.76      0.84      1204
                      Health & Beauty       0.96      0.31      0.47       753
                        Home & Living       0.40      0.97      0.57      1237
                      Home Appliances       1.00      0.00      0.01       293
                    Mobiles & Tablets       0.88      0.90      0.89      1431
TV, Audio / Video, Gaming & Wearables       0.88      0.45      0.60       474
         Watches Sunglasses Jewellery       0.97      0.80      0.88       879

     

In [24]:
print(f'Relative Error Ratio (One vs Rest / Baseline): {relative_error_ratio_baseline_stemmed}')
print(f'Relative Error Ratio (One vs One / Baseline): {relative_error_ratio_baseline_ovo}')
print(f'Relative Error Ratio (Crammer and Singer / Baseline): {relative_error_ratio_baseline_cs}')
print(f'Relative Error Ratio (Bigram OvR / Baseline): {relative_error_ratio_baseline_bigram}')
print(f'Relative Error Ratio (Bigram OvO / Baseline): {relative_error_ratio_baseline_bigram_ovo}')
print(f'Relative Error Ratio (Bigram Crammer and Singer / Baseline): {relative_error_ratio_baseline_bigram_cs}')
print(f'Relative Error Ratio (Poly OvO / Baseline): {relative_error_ratio_baseline_poly_ovo}')
print(f'Relative Error Ratio (Poly OvR / Baseline): {relative_error_ratio_baseline_poly}')
print(f'Relative Error Ratio (Poly Crammer and Singer / Baseline): {relative_error_ratio_baseline_poly_cs}')


Relative Error Ratio (One vs Rest / Baseline): 1.0241448692152917
Relative Error Ratio (One vs One / Baseline): 4.295774647887323
Relative Error Ratio (Crammer and Singer / Baseline): 1.0241448692152917
Relative Error Ratio (Bigram OvR / Baseline): 0.9134808853118715
Relative Error Ratio (Bigram OvO / Baseline): 8.22736418511066
Relative Error Ratio (Bigram Crammer and Singer / Baseline): 0.8812877263581482
Relative Error Ratio (Poly OvO / Baseline): 4.269617706237423
Relative Error Ratio (Poly OvR / Baseline): 1.0040241448692158
Relative Error Ratio (Poly Crammer and Singer / Baseline): 0.9879275653923542


In [25]:
# title_input = input("Enter product title: ")
# title_input_list = [title_input]
# X = vectorizer.transform(title_input_list)
# y_pred = best_svm.predict(X)
# predicted_label = label_encoder.inverse_transform(y_pred)
# print(predicted_label)
